In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pk
from Symbol import Symbol
from multiprocessing import Pool
import time

In [2]:
def load_debbuger(path,y):
    print(path+y)
    return pk.load(open(path+y,'rb'))

In [51]:
class Merger:
    def string_to_epoch(timestamp,arr, n):
        [date,t]=timestamp.split()
        date=[int(x) for x in date.split('-')]
        t=[int(x) for x in t.split(':')]

        if((date[1]<=3 and date[2]<=29 and t[0]<=2 and t[1]<=0)):
            tmz='CET'
        elif(date[1]>=10 and date[2]>=29 and t[0]>=3 and t[1]>=0):
            tmz='CET'
        elif(date[1]>=10 and date[2]>=29):
            try:
                prev=arr[n-12]
                [pdate,ptime]=prev.split()
                pdate=[int(x) for x in pdate.split('-')]
                ptime=[int(x) for x in ptime.split(':')]
                if(t[0]==ptime[0]):
                    tmz='CET'
                else:
                    tmz='CEST'
            except:
                futr=arr[n+12]
                [fdate,ftime]=prev.split()
                fdate=[int(x) for x in fdate.split('-')]
                ftime=[int(x) for x in ftime.split(':')]
                if(t[0]==ftime[0]):
                    tmz='CEST'
                else:
                    tmz='CET'
        else:
            tmz='CEST'


        return int(time.mktime(time.strptime(timestamp+' '+tmz,'%Y-%m-%d %H:%M %Z')))
    def load(path):
        files=[y for y in os.listdir(path)  if '.pkl' in y]
        objects=[pk.load(open(path+y,'rb'))[0] for y in files]
        data=[[[pd.DataFrame(x.data).transpose(),x.tick,pd.DataFrame(x.data).transpose().index]for x in y] for y in objects]
        datadic={}
        for file in data:
            for symbol in file:
                index=[Merger.string_to_epoch(x,symbol[0].index.tolist(),n) 
                       for x,n in zip(symbol[0].index.tolist(), 
                                      [n for n in range(len(symbol[0].index.tolist()))])]
                symbol[0].index=index
                if(symbol[1] not in datadic.keys()):
                    datadic[symbol[1]]=[symbol[0]]
                else:
                    datadic[symbol[1]]+=[symbol[0]]
        return datadic
    def merge_ticker(arg):
        [arr,ticker]=arg
        columns=[ticker+' '+x.replace('1.','').replace('2.','').replace('3.','').replace('4.','').replace('5.','').replace(' M','M')
                 for x in arr[0].columns]
        df=pd.DataFrame(columns=columns)
        for file in arr:
                for index in file.index:
                    if(index not in df.index):
                        df=df.append(pd.DataFrame(data=[file.loc[index].tolist()],index=[index],columns=columns))
        print(True in df.duplicated())
        return df.sort_index()
    def gen_df(path,fillna=None):
        startt=time.time()
        print('loading')
        data = Merger.load(path)
        tickers=list(data.keys())
        ltime=time.time()
        print(ltime-startt)
        print('merging tickers')
        with Pool(4) as p:
            dfs=p.map(Merger.merge_ticker,zip([data[ticker] for ticker in tickers],tickers))
        print(time.time()-ltime)
        if(fillna==None):
            fillna=-10000
        maindf=dfs[0]
        for df in dfs[1:]:
            for column in df.columns:
                maindf[column]=df[column]
        return maindf.fillna(fillna).astype(float)

In [52]:
dataframes=Merger.gen_df('./data/')

loading
7.489137411117554
merging tickers
False
False
False
False
False
False
False
False
False
18.208171606063843


In [53]:
dataframes

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,TSLA MACD_Hist,TSLA MACD,TSLA MACD_Signal,AAPL open,AAPL high,...,F MACD,F MACD_Hist,HPQ open,HPQ high,HPQ low,HPQ close,HPQ volume,HPQ MACD,HPQ MACD_Signal,HPQ MACD_Hist
1582539600,831.4800,834.4992,829.24,834.4700,91089.0,-8.6496,-1.4541,-7.1954,298.4000,299.1200,...,-10000.0000,-10000.0000,22.090,22.1200,22.0900,22.115,77860.0,-0.0515,-0.0466,-0.0048
1582539900,834.5000,835.5800,831.60,831.6900,88094.0,-8.6334,-0.9075,-7.7259,299.0598,299.3300,...,-10000.0000,-10000.0000,22.115,22.1150,22.0401,22.050,370036.0,-0.0494,-0.0479,-0.0015
1582540200,832.0400,837.3900,831.61,834.0000,95825.0,-8.5724,-0.3989,-8.1734,298.9800,299.8867,...,-10000.0000,-10000.0000,22.050,22.1000,22.0400,22.065,208738.0,-0.0497,-0.0493,-0.0003
1582540500,834.0000,836.0000,832.51,833.1300,82419.0,-8.1803,0.2399,-8.4202,299.1500,299.5000,...,-10000.0000,-10000.0000,22.065,22.0650,21.9900,22.000,195714.0,-0.0518,-0.0510,-0.0008
1582540800,833.2503,841.5000,833.12,839.5079,173332.0,-7.6089,0.8527,-8.4617,299.0500,300.7500,...,-10000.0000,-10000.0000,22.010,22.0699,22.0085,22.060,188129.0,-0.0513,-0.0521,0.0008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586353200,545.4900,546.3400,544.19,545.2000,90504.0,-0.0221,0.2939,0.3161,265.4400,265.5000,...,0.0035,0.0051,-10000.000,-10000.0000,-10000.0000,-10000.000,-10000.0,-10000.0000,-10000.0000,-10000.0000
1586353500,545.0500,546.5250,543.31,546.4200,122847.0,-0.2081,0.1625,0.3706,264.6100,265.1000,...,0.0013,0.0068,-10000.000,-10000.0000,-10000.0000,-10000.000,-10000.0,-10000.0000,-10000.0000,-10000.0000
1586353800,546.2000,548.9400,545.38,548.0178,151727.0,-0.2868,0.0818,0.3686,264.9000,265.9200,...,0.0011,0.0079,-10000.000,-10000.0000,-10000.0000,-10000.000,-10000.0,-10000.0000,-10000.0000,-10000.0000
1586354100,549.0000,550.4000,547.70,548.1201,227643.0,-0.1214,0.2344,0.3557,265.8500,267.3700,...,0.0025,0.0087,-10000.000,-10000.0000,-10000.0000,-10000.000,-10000.0,-10000.0000,-10000.0000,-10000.0000


In [45]:
def merge(dfs,fillna=None):
    if(fillna==None):
        fillna=-10000
    maindf=dfs[0]
    for df in dfs[1:]:
        for column in df.columns:
            maindf[column]=df[column]
    return maindf.fillna(fillna).astype(float)

In [46]:
test=merge(dataframes)

In [47]:
True in test.index.duplicated()

False

In [49]:
test['TSLA  open'].count

<bound method Series.count of 1582539600    831.4800
1582539900    834.5000
1582540200    832.0400
1582540500    834.0000
1582540800    833.2503
                ...   
1586353200    545.4900
1586353500    545.0500
1586353800    546.2000
1586354100    549.0000
1586354400    548.4400
Name: TSLA  open, Length: 2533, dtype: float64>

In [25]:
dataframes[7]

,HPQ open,HPQ high,HPQ low,HPQ close,HPQ volume,HPQ MACD,HPQ MACD_Signal,HPQ MACD_Hist
1582539600,22.0900,22.1200,22.0900,22.1150,77860,-0.0515,-0.0466,-0.0048
1582539900,22.1150,22.1150,22.0401,22.0500,370036,-0.0494,-0.0479,-0.0015
1582540200,22.0500,22.1000,22.0400,22.0650,208738,-0.0497,-0.0493,-0.0003
1582540500,22.0650,22.0650,21.9900,22.0000,195714,-0.0518,-0.0510,-0.0008
1582540800,22.0100,22.0699,22.0085,22.0600,188129,-0.0513,-0.0521,0.0008
...,...,...,...,...,...,...,...,...
1585230000,17.6500,17.6700,17.5650,17.6200,421192,0.0692,0.1049,0.1741
1585230300,17.6150,17.8800,17.6150,17.7950,454021,0.0924,0.1060,0.1984
1585230600,17.7950,17.8650,17.7700,17.8200,752229,0.1182,0.1109,0.2290
1585230900,17.8200,18.2100,17.8150,17.8500,1173358,0.1461,0.1143,0.2604
